## 1.목적
     의뢰사: Porto Seguro
     의뢰사 가치: 적정한 보험료 찾기
     (좋은운전자는 보험료를 적게, 나쁜운전자에게는 보험료를 많이 부과하기 위함)
     목표: 운전자가 내년에 자동차 보험 청구를 시작할 확률 예측
## 2.평가지표
     표준화 지니계수
## 3.데이터정의

# 소개

#### 1.데이터의 검사
#### 2.메타데이터
#### 3.기술통계
#### 4.불균형데이터처리
#### 5.데이터품질점검
#### 6.변수 엔지니어링
#### 7.변수 선택
#### 8.변수 스케일링

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns',100)

## 데이터 로드(Loading data)

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

## 데이터확인(Data at first sight)

데이터 설명:
* ** 유사 그룹화에 속하는 특징에는 피쳐 이름(예: in, reg, car, calcal)과 같이 **라는 태그가 붙는다.
* 피쳐 이름에는 이진 피쳐를 나타내는 포스트픽스 **bin**과 범주형 피쳐를 나타내는 **cat**가 포함된다. 
* 이러한 지정이 없는 특징 **은 연속 또는 서수**이다. 
* **-1*의 값은 해당 특징이 관찰 결과 ** 누락*임을 나타낸다. 
* **표적** 열은 해당 정책 보유자에 대한 청구 여부를 나타낸다.

In [ ]:
train.head()

In [ ]:
train.tail()

위의 결과로 다음을 확인할 수 있다. 

* 이진수 변수
* 범주 값이 정수인 범주형 변수
* 정수 또는 플로트 값이 있는 기타 변수
* -1이 결측값을 나타내는 변수
* 대상변수 및 ID변수

In [ ]:
train.shape

train 데이터에 중복된 행이 있는지 알아보자.

In [ ]:
train.drop_duplicates()
train.shape

중복된 행이 없음을 확인할 수 있다. 

In [ ]:
test.shape

test 세트에 변수가 하나 빠졌는데 이게 target_value이기 때문에 괜찮습니다.
이제 각 유형의 변수가 얼마나 많은지 호출해 봅시다.

그래서 나중에 우리는 14개의 범주형 변수에 대한 더미 변수를 만들 수 있다. 빈 변수는 이미 2진수여서 더밍이 필요하지 않다.

In [ ]:
train.info()

info()함수로  데이터 유형이 정수 또는 부동임을 알 수 있다. 데이터 집합에는 null 값이 없다. 결측값이 -1로 대체되기 때문

## Metadata
데이터 관리를 용이하게 하기 위해, 우리는 변수에 대한 메타 정보를 DataFrame에 저장할 것이다. 이는 분석, 시각화, 모델링 등을 위한 특정 변수를 선택하고자 할 때 유용할 것이다.

구체적으로 보관할 내용:
- **role**: 입력, ID, 대상
- **level**: 공칭, 간격, 서수, 이진
- **keep**: 참 또는 거짓
- **dtype**: int, float, str

In [ ]:
data = []
for f in train.columns:
    # Defining the role
    if f == 'target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
         
    # Defining the level
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif train[f].dtype == float:
        level = 'interval'
    elif train[f].dtype == int:
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if f == 'id':
        keep = False
    
    # Defining the data type 
    dtype = train[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

In [ ]:
meta

Example to extract all nominal variables that are not dropped(삭제되지 않은 모든 공칭 변수 추출 예제)

In [ ]:
meta[(meta.level == 'nominal') & (meta.keep)].index

Below the number of variables per role and level are displayed. (역할 및 수준당 변수 수 아래에 표시된다.)

In [ ]:
pd.DataFrame({'count' : meta.groupby(['role', 'level'])['role'].size()}).reset_index()

## 기술통계(Descriptive statistics)


우리는 또한 데이터 프레임에 기술 방법을 적용할 수 있다. 그러나, 범주형 변수와 ID 변수에 대한 평균, std를 계산하는 것은 별로 말이 되지 않는다. 우리는 나중에 시각적으로 범주형 변수를 탐구할 것이다.

메타 파일 덕분에 우리는 기술 통계를 계산할 변수를 쉽게 선택할 수 있다. 확실히 하기 위해, 우리는 데이터 유형별로 이것을 할 것이다

자료에 특성에 따른 변수 분류
:https://m.blog.naver.com/libido1014/120113775017

![pic](C:\Users\PAVILION\Pictures\pic.png)

### 간격변수(interval variables)


In [ ]:
v=meta[(meta.level=='interval') &(meta.keep)].index #행을 기준으로 인덱싱 :행의 이름들과 데이터형태 반환
train[v].describe() #describe:통계치반환

### 1.reg 변수
* ps_reg_03에만 누락된 값이 있음
* 변수 사이의 범위(최소 ~ 최대)가 다르다. 스케일링(예: StandardScaler)을 적용할 수 있지만, 사용할 분류기에 따라 다르다.

### 2.car 변수
* ps_car_12 및 ps_car_15에 누락된 값이 있음
* 다시, 범위가 달라져서 스케일링을 적용할 수 있다.데이터 스케일링이란 데이터 전처리 과정의 하나입니다.

  **(스케일링:전처리 방법중 하나로 데이터 스케일링을 해주는 이유는 데이터의 값이 너무 크거나 혹은 작은 경우에 모델 알고리즘 학습과정에서 0으로 수렴하거나 무한으로 발산해버릴 수 있기 때문**

### 3.calc 변수
* 누락된 값 없음
  이것은 최대치가 0.9이기 때문에 일종의 비율인 것 같다.
* 세 개의 calculate 변수는 모두 매우 유사한 분포를 가지고 있다.

* 전체적으로 보면 구간변수의 범위가 다소 작다는 것을 알 수 있다. 데이터를 익명화하기 위해 일부 변환(예: 로그)이 이미 적용되었는가?

### 순서형 변수(ordinal cariables)

In [ ]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
train[v].describe()

* ps_car_11에 누락값이 있다. 
* 다양한 범위를 처리하기 위해 스케일링을 적용할 수 있음

### 이진 변수(Binary variables)

In [ ]:
v = meta[(meta.level == 'binary') & (meta.keep)].index
train[v].describe()

- train 데이터의 평균값는 3.645%로 **강력 불균형(다른값은 대부분 10퍼센터 자리)** 
- 대부분의 변수에 대해 값이 0이라고 결론을 내릴 수 있다.

## 불균형 데이터 문제해결(Handling imbalanced classes)

위에서 언급한 바와 같이 target 갯수가  target=1 < target=0 이다  
이 문제를 해결하기 위한 두 가지 가능한 전략은 다음과 같다.
* target=1을 사용하여 레코드 오버샘플링 
* target=0으로 레코드 언더샘플링
위의 방법으로 데이터 비율을 맞추면 정밀도(precision)가 향상된다.

우리는 상당히 큰 트레이닝 세트를 가지고 있기 때문에 **undersampling**을 할 수 있다.(target=0이 다수이기 때문에 일분만 사용할 것임)


* 오버샘플링: 소수 클래스 데이터를 증가

* 언더샘플링 :다수 클래스 데이터에서 일부만 사용



(데이터 클래스 비율이 너무 차이가 나면(highly-imbalanced data) 단순히 우세한 클래스를 택하는 모형의 정확도가 높아지므로 모형의 성능판별이 어려워진다. 즉, 정확도(accuracy)가 높아도 데이터 갯수가 적은 클래스의 재현율(recall-rate)이 급격히 작아지는 현상이 발생할 수 있다.

이렇게 각 클래스에 속한 데이터의 갯수의 차이에 의해 발생하는 문제들을 비대칭 데이터 문제(imbalanced data problem)이라고 한다)

In [ ]:
desired_apriori=0.10

# Get the indices per target value(target당 인덱스 가져오기)
idx_0 = train[train.target == 0].index
idx_1 = train[train.target == 1].index

# Get original number of records per target value(target당 원본 레코드 수 가져오기)
nb_0 = len(train.loc[idx_0])
nb_1 = len(train.loc[idx_1])

# Calculate the undersampling rate and resulting number of records with target=0
  (목표값=0을 사용하여 언더샘플링 속도 및 결과 레코드 수 계산)
undersampling_rate = ((1-desired_apriori)*nb_1)/(nb_0*desired_apriori)
undersampled_nb_0 = int(undersampling_rate*nb_0)
print('Rate to undersample records with target=0: {}'.format(undersampling_rate))
print('Number of records with target=0 after undersampling: {}'.format(undersampled_nb_0))

# Randomly select records with target=0 to get at the desired a priori
  (원하는 우선순위를 얻기 위해 대상=0인 레코드를 무작위로 선택)
undersampled_idx = shuffle(idx_0, random_state=37, n_samples=undersampled_nb_0)

# Construct list with remaining indices(나머지 인덱스를 사용하여 목록 구성)
idx_list = list(undersampled_idx) + list(idx_1)

# Return undersample data frame(언더샘플 데이터 프레임 반환)
train = train.loc[idx_list].reset_index(drop=True)

## 데이터품질확인(Data Qulity Checks)

### 결측값확인(Checking missing values)

#### 결측값은 -1로 표현된다( Missing are represented as -1)

In [ ]:
vars_with_missing=[]

for f in train.columns:
    missings = train[train[f] == -1][f].count()
    if missings>0:
        vars_with_missing.append(f)
        missings_perc=missings/train.shape[0]
        
        print('Variable {} has records ({:.2%}) with missing values'.format(f,missings,missings_perc)
             )
        print('In total, ther are {} variables with missing values'.format(len(vars_with_missing)))

- **ps_car_03_cat과 ps_car_05_cat**는 값이 빠진 레코드의 비율이 크다. 이러한 변수를 제거해야한다. 
- 결측값이 있는 다른 범주형 변수에 대해서는 결측값 -1을 그대로 둘 수 있다.(????????)
- **ps_reg_03** (연속형) 18%에 대한 값이 누락됨 **평균**으로 대체한다.
- **ps_car_11**(범주형)에는 잘못된 값이 있는 5개의 레코드만 있다. 모드로 교체하십시오.
- **ps_car_12*** (연속형)은 값이 빠진 레코드가 1개뿐입니다. **평균**으로 대체한다.
- **ps_car_14*** (연속형) 모든 기록의 7%에 대한 값이 누락됨 **평균**으로 대체한다.

In [ ]:
# Dropping the variables with too many missing values
  (#결측값이 너무 많은 변수 삭제)
vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
train.drop(vars_to_drop, inplace=True, axis=1)
meta.loc[(vars_to_drop),'keep'] = False  # Updating the meta

# Imputing with the mean or mode
  (#평균 또는 모드로 임팩트하기)
mean_imp = Imputer(missing_values=-1, strategy='mean', axis=0)
mode_imp = Imputer(missing_values=-1, strategy='most_frequent', axis=0)
train['ps_reg_03'] = mean_imp.fit_transform(train[['ps_reg_03']]).ravel()
train['ps_car_12'] = mean_imp.fit_transform(train[['ps_car_12']]).ravel()
train['ps_car_14'] = mean_imp.fit_transform(train[['ps_car_14']]).ravel()
train['ps_car_11'] = mode_imp.fit_transform(train[['ps_car_11']]).ravel()

### 범주형 변수의 카디널리티 확인
카디널리티(Cardinity)는 **변수에서 서로 다른 값의 수**를 말한다. 나중에 범주형 변수로부터 더미 변수를 만들 것이므로, 많은 뚜렷한 값을 가진 변수가 있는지 확인할 필요가 있다. 우리는 이러한 변수들이 많은 더미 변수를 야기할 수 있기 때문에 다르게 다루어야 한다.

더미변수:더미변수는 반드시 0과 1이다.
1.남자:1 여자:2 => 0,1
2.학년의 경우 더미변수의 개수는 2개
-1학년여부:1학년=1,2~3학년=0
-2학년여부:2학년=1 1학년과 3학년 0

--------------------------------------------------------------------------------------

* 더미변수: 더미변수는 범주형 변수를 연속형 변수로 변환한 것인데, 정확히 따지자면 연속형 변수"스럽게" 만든 것

* 더미변수를 만드는 이유: 범주형 변수로는 사용할 수 없고 연속형 변수로만 가능한 분석기법을 사용할 수 있게 해준다(ex:선형회귀분석)

In [ ]:
v= meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    dist_values=train[f].value_counts().shape[0]
    print('Variable {} has {} distinct values'.format(f,dist_values))

* ps_car_11_cat 만이 여전히 합리적이지만 많은 뚜렷한 값을 가지고 있다.  

In [ ]:
#스무딩: 범주형 평균과 이전 평균의 균형을 맞추기 위한 스무딩 효과 /데이터 스무딩은 데이터에서 원치 않는 잡음이나 동작을 제거하는 기법
# Script by https://www.kaggle.com/ogrellier
# Code: https://www.kaggle.com/ogrellier/python-target-encoding-for-categorical-features
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [ ]:
train_encoded, test_encoded = target_encode(train["ps_car_11_cat"], 
                             test["ps_car_11_cat"], 
                             target=train.target, 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)
    
train['ps_car_11_cat_te'] = train_encoded
train.drop('ps_car_11_cat', axis=1, inplace=True)
meta.loc['ps_car_11_cat','keep'] = False  # Updating the meta
test['ps_car_11_cat_te'] = test_encoded
test.drop('ps_car_11_cat', axis=1, inplace=True)

## 탐색적 데이터 시각화(Exploratory Data Visualization)

## 범주형변수(Categorical variables)

범주형 변수와 target=1 의 고객 비율을 보자

In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    plt.figure()
    fig, ax = plt.subplots(figsize=(20,10))
    # Calculate the percentage of target=1 per category value
    cat_perc = train[[f, 'target']].groupby([f],as_index=False).mean()
    cat_perc.sort_values(by='target', ascending=False, inplace=True)
    # Bar plot
    # Order the bars descending on target mean
    sns.barplot(ax=ax, x=f, y='target', data=cat_perc, order=cat_perc[f])
    plt.ylabel('% target', fontsize=18)
    plt.xlabel(f, fontsize=18)
    plt.tick_params(axis='both', which='major', labelsize=18)
    plt.show();

**결측값**  을 변수에서 볼 수 있듯이, 결측값을 예를 들어 모드로 대체하는 대신 별도의 범주값으로 유지하는 것이 좋다. (결측값이 많은 고객들은 보험금 청구(target=1)를 요구할 확률이 훨씬 더 높은 것으로 보인다.??)


### 간격변수 (Interval variable)

간격 변수 간의 상관 관계 확인. Heat map은 변수들 사이의 상관 관계를 시각화하는 좋은 방법이다.

In [ ]:
def corr_heatmap(v):
    correlations = train[v].corr()

    # Create color map ranging between two colors
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(correlations, cmap=cmap, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .75})
    plt.show();
    
v = meta[(meta.level == 'interval') & (meta.keep)].index
corr_heatmap(v)
                

변수들 사이에는 강한 상관관계가 있다.
- ps_reg_02 및 ps_reg_03(0.7)
- ps_car_12 및 ps_car13(0.67)
- ps_car_12 및 ps_car14(0.58)
- ps_car_13 및 ps_car15(0.67)

Seaborn은 변수들 사이의 (선형적인) 관계를 시각화할 수 있는 몇 가지 유용한 그래프 를 가지고 있다. 우리는 변수들 사이의 관계를 시각화하기 위해 *쌍구*를 사용할 수 있다. 하지만 열 지도에서 이미 제한된 수의 상관 변수를 보여 주었기 때문에, 각각의 고도로 상관관계가 있는 변수를 개별적으로 살펴보도록 하겠다.
**NOTE**: 속도를 높이기 위해 train 데이터의 샘플을 채취한다.

In [ ]:
s=train.sample(frac=0.1)


#### ps_reg_02 및 ps_reg_03
회귀선이 보여주듯이 이들 변수 사이에는 선형 관계가 있다. *hue* 매개변수 덕분에 우리는 target=0과 target=1의 회귀선이 같다는 것을 알 수 있다.

In [ ]:
sns.lmplot(x='ps_reg_02', y='ps_reg_03', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()
#lmplot:상관관계를 바로 알수있는 그래프
#hue :X의 유무를 통해 회귀선을 알 수 있다. 

#### ps_car_12 and ps_car_13

In [ ]:
sns.lmplot(x='ps_car_12', y='ps_car_13', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3   
                                                                                           }
          )
plt.show()

#### ps_car_12 and ps_car_14

In [ ]:
sns.lmplot(x='ps_car_12', y='ps_car_14', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

#### ps_car_13 and ps_car_15

In [ ]:
sns.lmplot(x='ps_car_15', y='ps_car_13', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

어떤 상관 변수를 유지할 것인지 어떻게 결정할 수 있는가? 변수에 대한 ** 주성분 분석(PCA) ** 을 수행하여 치수를 줄일 수 있다. 올스테이트 클레임 심각성 대회에서 나는 그것을 하기 위해 [이 kernel](https://www.kaggle.com/bertcarremans/reducing-number-of-numerical-features-with-pca)를 만들었다. 그러나 상관 변수의 수가 다소 적기 때문에 우리는 모델이 무거운 것을 하도록 할 것이다

### 순서형변수의 상관계수 확인(Checking the correlations between ordinal variables)

In [ ]:
v= meta[(meta.level == 'ordinal')&(meta.keep)].index
corr_heatmap(v)


순서형변수의 경우 우리는 많은 상관관계를 보지 못한다. 반면에 우리는 target으로 그룹화할 때 분포가 어떤지 살펴볼 수 있다.

## 변수 엔지니어링(Feature engineering)


### 더미 변수 작성
범주형 변수의 값은 어떤 순서나 크기를 나타내지 않는다. 따라서 우리는 그것을 다루기 위해 더미 변수를 만들 수 있다. 이 정보가 원래 변수의 범주에 대해 생성된 다른 더미 변수에서 도출될 수 있기 때문에 우리는 **첫 번째 더미 변수**를 떨어뜨린다.(???????????????????)

* 더미변수: 더미변수는 범주형 변수를 연속형 변수로 변환한 것인데, 정확히 따지자면 연속형 변수"스럽게" 만든 것

* 더미변수를 만드는 이유: 범주형 변수로는 사용할 수 없고 연속형 변수로만 가능한 분석기법을 사용할 수 있게 해준다(ex:선형회귀분석)

In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index
print('Before dummification we have {} variables in train'.format(train.shape[1]))
train = pd.get_dummies(train, columns=v, drop_first=True)
print('After dummification we have {} variables in train'.format(train.shape[1]))

In [ ]:
따라서 더미 변수를 생성하면 52개의 변수가 훈련 세트에 추가된다

### 교호작용 변수 생성(Creating interaction variables)

In [ ]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
interactions = pd.DataFrame(data=poly.fit_transform(train[v]), columns=poly.get_feature_names(v))
interactions.drop(v, axis=1, inplace=True)  # Remove the original columns
# Concat the interaction variables to the train data
print('Before creating interactions we have {} variables in train'.format(train.shape[1]))
train = pd.concat([train, interactions], axis=1)
print('After creating interactions we have {} variables in train'.format(train.shape[1]))

이것은 train 데이터에 추가적인 상호작용 변수를 추가한다. *get_feature_names* 방법 덕분에 열 이름을 다음 항목에 할당할 수 있음 
새로운 변수

## 변수선택(Feature selection)

### 분산이 낮거나 0인 피쳐 제거(Removing features with low or zero variance)


##### 변수의 분산이 거의 0인 경우
예를 들어 데이터 1000개중에 990개에서 변수 A열의 값이 0이고, 나머지 10개가 10이라면 
이 변수 A는 행을 구분하는데 거으이 의미가 없다. 이런변수는 분산이 거의 0인경우 분산이 겅의 0인 변수는
제거해준다. 

https://jkook.tistory.com/19

개인적으로, 나는 분류자 알고리즘이 유지할 특징을 선택하도록 하는 것을 선호한다. 그러나 우리가 스스로 할 수 있는 일이 하나 있다. 그것은 전혀 또는 매우 낮은 분산이 있는 특징들을 제거하는 것이다. Sklearn은 이를 위한 편리한 방법을 가지고 있다: **변성임계값**. 기본적으로 분산 없이 형상을 제거한다. 이는 이전 단계에서 무분산 변수가 없는 것을 보았기 때문에 이 경기에는 적용되지 않을 것이다. 그러나 1% 미만의 분산을 가진 형상을 제거한다면 31개의 변수를 제거하게 될 것이다

In [ ]:
selector = VarianceThreshold(threshold=.01)
selector.fit(train.drop(['id', 'target'], axis=1)) # Fit to train without id and target variables

f = np.vectorize(lambda x : not x) # Function to toggle boolean array elements

v = train.drop(['id', 'target'], axis=1).columns[f(selector.get_support())]
print('{} variables have too low variance.'.format(len(v)))
print('These variables are {}'.format(list(v)))

만약 우리가 분산에 근거하여 선택한다면, 우리는 오히려 많은 변수들을 잃게 될 것이다. 그런데 워낙 변수가 많지 않기 때문에 selector가 선택하도록 하겠다. 변수가 많은 데이터 세트의 경우, 이는 처리 시간을 줄일 수 있다.

스클리어엔 다른 [기능 선택 방법](http://scikit-learn.org/stable/modules/feature_selection.html)도 함께 제공된다. 이러한 방법 중 하나는 다른 분류자가 최상의 형상을 선택하고 이를 계속하도록 하는 *SelectFromModel*이다. 아래는 랜덤 포레스트로 어떻게 하는지 보여줄게.

In [ ]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train['target']

feat_labels = X_train.columns

rf = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)

rf.fit(X_train, y_train)
importances = rf.feature_importances_

indices = np.argsort(rf.feature_importances_)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,feat_labels[indices[f]], importances[indices[f]]))


SelectFromModel을 사용하여 사용할 사전 적합 분류기와 형상 가져오기의 임계값을 지정할 수 있다. *get_support* 방법으로 열차 데이터의 변수 수를 제한할 수 있다

In [ ]:
sfm = SelectFromModel(rf, threshold='median', prefit=True)
print('Number of features before selection: {}'.format(X_train.shape[1]))
n_features = sfm.transform(X_train).shape[1]
print('Number of features after selection: {}'.format(n_features))
selected_vars = list(feat_labels[sfm.get_support()])


In [ ]:
train = train[selected_vars + ['target']]

## 변수 스케일링(Feature scaling)
앞에서 언급했듯이, 우리는 훈련 데이터에 표준 스케일링을 적용할 수 있다. 어떤 selector들은 이것이 완성되었을 때 더 나은 성능을 발휘한다.

In [ ]:
scaler = StandardScaler()
scaler.fit_transform(train.drop(['target'], axis=1))